## 1. Importaciones y Configuración

Importamos las bibliotecas necesarias:
- **Annotated**: Para anotaciones de tipo ricas
- **AzureOpenAIChatClient**: Cliente de Azure OpenAI
- **stdio_server**: Servidor MCP basado en stdio
- **anyio**: Biblioteca de concurrencia (alternativa a asyncio)

In [ ]:
from typing import Annotated
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from dotenv import load_dotenv
import os
import anyio

load_dotenv()

print("✅ Importaciones completadas")
print("📡 Preparado para crear servidor MCP")

## 2. Definición de Herramientas (Tools)

### Herramientas del Restaurante

Creamos funciones que representan las capacidades de nuestro agente de restaurante:

#### get_specials()
- Retorna los especiales del día del menú
- No requiere parámetros
- Usa `Annotated` para proporcionar descripción rica

#### get_item_price()
- Retorna el precio de un elemento del menú
- Parámetro: `menu_item` (nombre del plato)
- En producción, consultaría una base de datos

### Anotaciones Importantes:

```python
Annotated[str, "Descripción"]
```

El segundo elemento de `Annotated` proporciona metadatos que:
- Ayudan al LLM a entender cuándo usar la función
- Se exponen a través del protocolo MCP
- Facilitan el discovery de herramientas por clientes

In [ ]:
def get_specials() -> Annotated[str, "Retorna los especiales del menú."]:
    """Obtiene los especiales del día del restaurante."""
    return """
        Sopa Especial: Clam Chowder
        Ensalada Especial: Cobb Salad
        Bebida Especial: Chai Tea
        """

def get_item_price(
    menu_item: Annotated[str, "El nombre del elemento del menú."],
) -> Annotated[str, "Retorna el precio del elemento del menú."]:
    """Obtiene el precio de un elemento específico del menú."""
    # En un caso real, esto consultaría una base de datos
    return "$9.99"

print("✅ Herramientas del restaurante definidas")
print("🍽️  Disponibles: get_specials, get_item_price")

# Probar las funciones
print("\n🧪 Prueba de funciones:")
print(get_specials())
print(f"Precio de 'Pasta': {get_item_price('Pasta')}")

## 3. Creación del Agente con Tools

### Configuración del Agente:

El agente se configura con:
- **name**: "RestaurantAgent" - Identificador del agente
- **description**: Propósito del agente (importante para MCP discovery)
- **tools**: Lista de funciones disponibles

### Características del Agente:

1. **Especialización**: Enfocado en responder preguntas sobre el menú
2. **Tool Access**: Puede llamar a `get_specials` y `get_item_price`
3. **Autonomía**: Decide cuándo usar cada herramienta
4. **Interoperabilidad**: Puede exponerse vía MCP

### Flujo de Decisión del Agente:

```
Usuario: "¿Cuáles son los especiales?"
  ↓
Agente analiza la consulta
  ↓
Agente decide: Necesito llamar get_specials()
  ↓
Función se ejecuta y retorna resultados
  ↓
Agente formula respuesta al usuario
```

In [ ]:
# Crear un agente con herramientas
agent = AzureOpenAIChatClient(
    credential=AzureCliCredential(),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment_name=os.getenv("MODEL"),
).create_agent(
    name="RestaurantAgent",
    description="Responde preguntas sobre el menú del restaurante.",
    tools=[get_specials, get_item_price],
)

print("✅ RestaurantAgent creado")
print("🤖 Nombre: RestaurantAgent")
print("📋 Descripción: Responde preguntas sobre el menú")
print("🛠️  Herramientas: 2 disponibles")

## 4. Exposición como Servidor MCP

### ¿Qué hace `.as_mcp_server()`?

Este método:
1. **Envuelve el agente** en un servidor MCP estándar
2. **Expone las herramientas** siguiendo el protocolo MCP
3. **Maneja la comunicación** con clientes MCP
4. **Traduce mensajes** entre el protocolo MCP y el agente

### Protocolo MCP:

El servidor MCP puede:
- **Listar herramientas**: Responder a `tools/list`
- **Ejecutar herramientas**: Manejar `tools/call`
- **Proporcionar esquemas**: Describir parámetros y tipos
- **Manejar errores**: Respuestas estándar de error

### Ventajas de MCP:

1. **Estandarización**: Protocolo común entre diferentes sistemas
2. **Reutilización**: El mismo agente en múltiples contextos
3. **Discovery**: Clientes pueden descubrir capacidades automáticamente
4. **Interoperabilidad**: Funciona con cualquier cliente compatible con MCP

In [ ]:
# Exponer el agente como un servidor MCP
server = agent.as_mcp_server()

print("✅ Servidor MCP creado")
print("📡 Protocolo: Model Context Protocol (MCP)")
print("🔌 Transporte: stdio (standard input/output)")
print("\n💡 El servidor puede ahora ser consumido por:")
print("   - Claude Desktop")
print("   - VS Code con extensiones MCP")
print("   - Otros clientes compatibles con MCP")

## 5. Configuración del Servidor stdio

### ¿Qué es stdio_server?

`stdio_server` es un transport layer para MCP que:
- Lee mensajes desde **stdin** (standard input)
- Escribe respuestas a **stdout** (standard output)
- Permite comunicación bidireccional
- Es ideal para integración con herramientas de línea de comandos

### Flujo de Comunicación:

```
Cliente MCP
    ↓ (escribe a stdin)
    ↓ {"method": "tools/list"}
stdio_server (lee de stdin)
    ↓ (procesa)
Servidor MCP
    ↓ (retorna respuesta)
stdio_server (escribe a stdout)
    ↓ {"result": [{"name": "get_specials", ...}]}
    ↓
Cliente MCP (lee de stdout)
```

### Inicialización del Servidor:

```python
server.run(read_stream, write_stream, initialization_options)
```

Parámetros:
- **read_stream**: Stream de entrada (stdin)
- **write_stream**: Stream de salida (stdout)
- **initialization_options**: Opciones de configuración del servidor

In [ ]:
from mcp.server.stdio import stdio_server

async def run():
    """Ejecuta el servidor MCP con transporte stdio."""
    async def handle_stdin():
        async with stdio_server() as (read_stream, write_stream):
            await server.run(
                read_stream, 
                write_stream, 
                server.create_initialization_options()
            )

    await handle_stdin()

print("✅ Función de servidor stdio definida")
print("\n⚠️  NOTA: En un script Python normal, ejecutarías:")
print("   if __name__ == '__main__':")
print("       anyio.run(run)")
print("\n📝 En un notebook, no podemos ejecutar el servidor stdio")
print("   porque requiere control de stdin/stdout.")

## 6. Prueba del Agente (Sin MCP)

Aunque no podemos ejecutar el servidor stdio en un notebook, podemos probar el agente directamente:

In [ ]:
# Probar el agente directamente (sin MCP)
async def test_agent():
    print("🧪 Probando el agente directamente...\n")
    
    # Test 1: Preguntar por especiales
    print("❓ Test 1: ¿Cuáles son los especiales de hoy?\n")
    response1 = await agent.run("What are today's specials?")
    print("💬 Respuesta:")
    print(response1.text)
    print("\n" + "="*60 + "\n")
    
    # Test 2: Preguntar por precio
    print("❓ Test 2: ¿Cuánto cuesta la sopa especial?\n")
    response2 = await agent.run("How much does the special soup cost?")
    print("💬 Respuesta:")
    print(response2.text)
    print("\n" + "="*60 + "\n")
    
    # Test 3: Pregunta combinada
    print("❓ Test 3: Dime los especiales y sus precios\n")
    response3 = await agent.run("Tell me the specials and their prices")
    print("💬 Respuesta:")
    print(response3.text)

await test_agent()

## 7. Configuración para Claude Desktop

### Cómo usar este agente con Claude Desktop:

1. **Guardar el script** como `mcp_agent.py`

2. **Crear archivo de configuración** de Claude Desktop:

**En Windows**: `%APPDATA%\Claude\claude_desktop_config.json`

**En macOS**: `~/Library/Application Support/Claude/claude_desktop_config.json`

3. **Agregar configuración MCP**:

```json
{
  "mcpServers": {
    "restaurant": {
      "command": "python",
      "args": [
        "c:\\ruta\\a\\mcp_agent.py"
      ],
      "env": {
        "AZURE_OPENAI_ENDPOINT": "tu-endpoint",
        "MODEL": "tu-modelo"
      }
    }
  }
}
```

4. **Reiniciar Claude Desktop**

5. **Usar el agente**: Claude puede ahora llamar a las herramientas del restaurante

In [ ]:
# Generar configuración de ejemplo
import json
from pathlib import Path

config = {
    "mcpServers": {
        "restaurant": {
            "command": "python",
            "args": [
                str(Path.cwd() / "mcp_agent.py")
            ],
            "env": {
                "AZURE_OPENAI_ENDPOINT": "https://tu-recurso.openai.azure.com/",
                "MODEL": "gpt-4o"
            }
        }
    }
}

print("📝 Configuración de ejemplo para Claude Desktop:")
print("="*60)
print(json.dumps(config, indent=2))
print("="*60)
print("\n💡 Guarda esto en el archivo de configuración de Claude Desktop")

## 8. Integración con VS Code

### Uso con extensiones MCP en VS Code:

Algunas extensiones de VS Code soportan MCP. Para usarlas:

1. **Instalar extensión compatible con MCP**

2. **Configurar en settings.json**:

```json
{
  "mcp.servers": [
    {
      "name": "restaurant",
      "command": "python",
      "args": ["c:\\ruta\\a\\mcp_agent.py"]
    }
  ]
}
```

3. **Usar el servidor** desde la extensión

### Debugging del Servidor:

Para debugging, puedes agregar logging:

```python
import logging
logging.basicConfig(
    level=logging.DEBUG,
    filename='mcp_server.log'
)
```

Esto registrará todas las operaciones del servidor en un archivo.

## 9. Arquitectura de Producción

### Sistema Completo con MCP:

```python
# mcp_restaurant_service.py

import logging
from pathlib import Path

# Configurar logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('mcp_server.log'),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(__name__)

class RestaurantDatabase:
    """Simula una base de datos de restaurante."""
    
    def __init__(self):
        self.menu = {
            "Clam Chowder": 8.99,
            "Cobb Salad": 12.99,
            "Chai Tea": 3.99,
            "Burger": 14.99,
            "Pasta": 16.99
        }
        self.specials = ["Clam Chowder", "Cobb Salad", "Chai Tea"]
    
    def get_specials(self) -> list[dict]:
        return [
            {"name": item, "price": self.menu[item]}
            for item in self.specials
        ]
    
    def get_price(self, item: str) -> float | None:
        return self.menu.get(item)

# Instancia global de la base de datos
db = RestaurantDatabase()

def get_specials() -> Annotated[str, "Returns menu specials"]:
    logger.info("get_specials called")
    specials = db.get_specials()
    result = "\n".join(
        f"{s['name']}: ${s['price']:.2f}" 
        for s in specials
    )
    return result

def get_item_price(
    menu_item: Annotated[str, "Menu item name"]
) -> Annotated[str, "Returns item price"]:
    logger.info(f"get_item_price called with: {menu_item}")
    price = db.get_price(menu_item)
    if price:
        return f"${price:.2f}"
    else:
        return f"Item '{menu_item}' not found in menu"

# ... resto del código del servidor
```

## 10. Análisis y Conclusiones

### Ventajas del Patrón MCP:

1. **Interoperabilidad**:
   - Un solo agente, múltiples clientes
   - Protocolo estándar
   - Facilita integración

2. **Reutilización**:
   - Escribe una vez, usa en muchos lugares
   - No necesitas adaptar código para cada cliente
   - Mantenimiento centralizado

3. **Discovery Automático**:
   - Clientes pueden descubrir herramientas
   - Esquemas auto-generados
   - Documentación incluida

4. **Separación de Responsabilidades**:
   - Lógica de negocio separada del transporte
   - Fácil testing
   - Arquitectura limpia

### Casos de Uso Prácticos:

1. **Integración con IDEs**:
   - Agentes disponibles en VS Code, Cursor, etc.
   - Autocompletado inteligente
   - Asistencia contextual

2. **Claude Desktop Extensions**:
   - Extender capacidades de Claude
   - Acceso a APIs privadas
   - Herramientas personalizadas

3. **Microservicios de IA**:
   - Agentes como servicios
   - Orquestación de múltiples agentes
   - Arquitectura distribuida

4. **Automatización Empresarial**:
   - Agentes para tareas específicas
   - Integración con sistemas legacy
   - Workflows inteligentes

### Comparación con Otras Arquitecturas:

| Característica | MCP | REST API | Function Calling |
|---|---|---|---|
| Estandarización | ✅ Alto | ⚠️ Varía | ❌ No estándar |
| Interoperabilidad | ✅ Excelente | ⚠️ Media | ❌ Limitada |
| Discovery | ✅ Automático | ⚠️ Manual | ⚠️ Requiere doc |
| Complejidad | ⚠️ Media | ✅ Simple | ✅ Simple |
| Ecosistema | ✅ Creciente | ✅ Maduro | ⚠️ Emergente |

### Mejores Prácticas:

1. **Documentación Rica**:
   - Usa descripciones claras en Annotated
   - Docstrings detallados
   - Ejemplos de uso

2. **Manejo de Errores**:
   - Retorna mensajes de error descriptivos
   - Valida inputs
   - Logging apropiado

3. **Testing**:
   - Prueba herramientas independientemente
   - Prueba integración MCP
   - Tests end-to-end con clientes

4. **Seguridad**:
   - Valida todos los inputs
   - Controla acceso a recursos
   - Audita operaciones

5. **Performance**:
   - Implementa caché cuando sea apropiado
   - Optimiza consultas a bases de datos
   - Monitorea latencia

### Limitaciones y Consideraciones:

1. **stdio Transport**:
   - No apto para alta concurrencia
   - Un cliente a la vez
   - Para producción, considerar WebSocket o HTTP

2. **Estado**:
   - servidores MCP suelen ser stateless
   - Si necesitas estado, implementa persistencia

3. **Escalabilidad**:
   - stdio no escala horizontalmente
   - Para múltiples clientes, usar transporte diferente

### Futuro de MCP:

- **Adopción creciente**: Más herramientas soportarán MCP
- **Extensiones del protocolo**: Nuevas capacidades
- **Mejores herramientas**: SDKs, debugging, monitoring
- **Integración cloud**: Azure, AWS, GCP
- **Ecosistema**: Marketplace de agentes MCP